<a href="https://colab.research.google.com/github/BURHANUDDIN51-PYTHON/fastai-under-the-hood/blob/main/01_mnist_tensors_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *
matplotlib.rc('image', cmap='Greys')
import torch

In [ ]:
# Download the mnist data (sampple)
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
(path/'train').ls()

In [ ]:
# Build the 3's and 7's Classifier
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()


# First build the classifier using the Similarity pixel algorithim

In [ ]:
# Get the image in the tensor form
three_tensors = [tensor(Image.open(o)) for o in threes]
seven_tensors = [tensor(Image.open(o)) for o in sevens]

In [ ]:
# Get the mean image of both the 3's and 7's
show_image(three_tensors[1])

In [ ]:
# Stack the images and calculate the mean image of 3 and 7
mean3 = (torch.stack(three_tensors).float()/255).mean(0)
mean7 = (torch.stack(seven_tensors).float()/255).mean(0)

In [ ]:
show_image(mean3), show_image(mean7)

In [ ]:
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255

valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

valid_3_tens.shape, valid_7_tens.shape



In [ ]:
# define the loss function
def mnist_distance(a,b): return (a-b).abs().mean(-1,-2)

In [ ]:
# define the funciton to check
def is_3(x): return mnist_distance(x, mean3) < mnist_distance(x, mean7)

The approch till yet is the similarity pixel approach


## Build the MNIST classification model from scratch


In [ ]:
# Get the data for the MNIST data
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path
path.ls()

In [ ]:
# Building the traing data
threes_data = torch.stack([
    tensor(Image.open(o)) for o in (path/'train'/'3').ls()
]).float() / 255
seven_data = torch.stack([
    tensor(Image.open(o)) for o in (path/'train'/'7').ls()
]).float() / 255

threes_data.shape, seven_data.shape

In [ ]:
# Build the x_train data
x_train = torch.cat([threes_data, seven_data]).view(-1,28*28)
y_train = tensor([1]*len(threes_data)+[0]*len(sevens)).unsqueeze(1)
dset = list(zip(x_train,y_train))

In [ ]:
x,y = dset[0]
x.shape,y

In [ ]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [ ]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [ ]:
weights = init_params((28*28,1))
bias = init_params(1)

def linear1(xb): return xb@weights + bias
preds = linear1(x_train)
preds

In [ ]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [ ]:
weights = init_params((28*28, 1))
bias = init_params(1)

In [ ]:
# create the dataloaders for the training and validation set
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256 )

In [ ]:
def calc_grad(xb, yb, model):
  pred = model(xb)
  loss = mnist_loss(pred, yb)
  loss.backward()

In [ ]:
def train_epoch(model, lr, params):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

In [ ]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [ ]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [ ]:
lr = 1.
params = weights,bias

for i in range(20):
    train_epoch(linear1, lr, params)
    print(validate_epoch(linear1), end=' ')